# 7월 17일

- 기업 쪽으로 간다면 redhat 계열 리눅스
- 보안 쪽으로 간다면 Ubuntu 계열 리눅스

## Snort 설치
### 설치


- 우분투에 설치하여 실습

- LAN 카드 이름이 이더넷0로 되어 있어야 한다.
    - 변경 작업 필요.
    - apt-get update
    - apt-get install net-tools
    - nano /etc/default/grub
    - GRUB_CMDLINE_LINUX="find_pressed=/preseed.cfg auto noprompt
    - priority=critical locale=en_US net.ifnames=0 biosdevname=0"
    - grub-mkconfig -o /boot/grub/grub.cfg
    - reboot

- Snort 설치
    - apt-get install -y snort  (설치)
    - snot-V    (버전 확인)



### Rule 생성 후 실행

1. snort 설치 후 관제 룰 생성
    - 환경 설정 파일: **snort.conf** (중요 파일)
    - rules 디렉터리에 룰 존재 --> 관제 룰 파일들 저장.
        - 사용자 생성 룰은 local.rules에 저장. --> nano local.rules
        - ICMP 패킷에 대해 잘 관제하는지 확인
            - alert icmp any any -> any any (msg:"Test Ping"; sid:100001;)
            - alert: 경고 발생 + 로그 생성
            - any any -> any any : 송신지 IP, Port -> 수신지 IP, Port

2. snort -T -c /etc/snort/snort.conf
    - Snort IDS 테스트
    - 정상적으로 돌아가는지 확인!
    - **Snort successfully validated the configuration** 문구가 뜨면 관제 시 문제 없음을 의미

    - Fatal Error, Quitting 문구가 뜨면 오류가 발생했음을 알 수 있음
    - 문법적 오류가 발생 시 문제 발생원인 확인 가능. 원인 또는 원인 근처를 알려준다.

3. snort -i eth0 -c /etc/snort/snort.conf
    - 생성된 룰 **적용** 후 IDS 실행
    - Snort를 침입 탐지 시스템(IDS) 모드로 실행하는 기본적인 명령
    - 실습: 칼리에서 메타로 ping test 시 관제가 되는지 확인

4. 로그 확인(관제 창)
    - snort -A console -c /etc/snort/snort.conf
    - 콘솔에서 실시간 로그 확인

    - tail -f /var/log/snort/snort.alert.fast
    - 파일로 저장된 로그 확인

# Snort Rule
- 오픈소스 **시그니처 기반 NIDS**
    - NIDS: 네트워크 기반
    - 오용탐지 방법!
- 준비된 규칙과 비교하여 침입 탐지 및 경로를 발생

- 시그니처 기반: 침입 탐지를 문자열로 판단하는 것
    - 패킷 데이터에서 악의적인 문자열을 탐지하여 침입 여부를 결정
    - 오늘날 침입 탐지 시스템의 대명사로 사용

- 네트워크 패킷을 수집하여 트래픽을 모니터링
- 준비된 규칙과 비교하여 침입 탐지 및 경로를 발생


### Suricata(수리카타)
- Snort의 단점을 개선하고 장점을 수용
    - 망의 규모가 클 때에는 Snort가 적합하지 않음
    - 수리카다는 멀티 코어 및 멀티 스레드를 지원하여 대용량 트래픽 실시간 처리(성능 향상)
    - Snort Rule 완변 호환 및 대부분의 기능 지원
    - 하드웨어 벤더의 개발 지원으로 하드웨어 가속 지원
    - 스크립트 언어(Lua) 지원



### Snort 시그니처 기반의 IDS의 Detection Rule 기본 구조 (Page 6)
- 두 부분으로 구성(헤더+옵션)
    - ## 헤더: 괄호가 나오기 전까지를 규칙 헤더
        - 헤더 내에서 3가지 부분으로 구성
            1. ### Action: alert와 같이 특정 패킷을 탐지했을 때 어떻게 행동할건지를 정함.
                - alert: 경고 + 로그
                - log: 패킷을 로그로 저장
                - pass: 패킷 무시
                - active: 경고 후 다른 동적 규칙 활성화
                - dynamic: acitve 옵션으로 활성화
                - drop: 패킷 차단 후 로그로 저장
                - reject: 
                    - TCP는 차단 및 로그 저장 후 세선 리셋(RST전송)
                    - UDP는 차단 및 로그 저장 후 ICMP 메시지 전송
                - sdrop: 패킷 차단. 로그 X
                - drop, reject, sdrop을 사용하려면 적어도 **인라인 모드**로 설치가 되어 있어야 한다.

            2. ### Protocol: 프로토콜 명시. 상위 계층 프로토콜은 TCP, UDP로 구분한다.



3. ### 트래픽 흐름 방향
- (Src IP) (Src Port) (Direction(<-->)) (Dst IP) (Dst Port)순서
    - IP
        - any: 모든 IP 주소
        - 1.1.1.1: 특정 IP 주소
        - [1.1.1.1,2.2.2.2]: 여러 IP 주소. 송신지 IP가 정해져있지 않은 상태. 여러 개 중 하나!
        - [1.1.1.1/24]: 특정 IP 주소
        - 변수: 가변적인 주소를 사용하기 위해 변수를 사용. 어느 회사에서 사용할지 모르기 때문에 변수를 설정한다.
                
    - Port
        - any: 모든 포트 번호
        - 80: 80번과 같은 특정 포트 주소
        - 1:1024: 1부터 1024번 범위
        - 80: 80번 이상 포트 번호
        - :1024: 1024 이하 포트 번호
        - !80: 80을 제외한 포트 번호
                
    - Direction
        - ->: 단방향
        - <>: 양방향(거의 안씀)



- ## 옵션: 괄호 부분을 규칙 옵션
    - 규칙 헤더에 해당하는 패킷 중 **특정 패턴(문자열: 시그니처)**을 정의해 놓은 영역.
- ### 옵션 종류
    - 일반 옵션
        - 규칙에 대한 정보를 제공하는 옵션
        - 검색하는 동안 어떤 영향도 미치지 않음
            - msg 
                - 규칙이 탐지될 경우 출력되는 메시지
                - 공격 유형과 정보를 기록
            - sid
                - 규칙 식별자로 모든 규칙은 반드시 식별 번호를 가짐
                - 예약된 식별자: 0 ~ 2,999,999
                - Local.rules에는 3,000,000이상부터 사용
            - rev
                - 규칙의 수정 버전을 나타냄
                - 규칙이 수정 시 1씩 증가
            - classtype
                - 규칙을 분류하는 옵션
                - 클래스 명은 ckassfication.config 파일에 정의
            - priority
                - 규칙의 우선순위 지정
                - 1~10까지의 수 사용, 숫자가 작을수록 높은 우선 순위를 가짐

        - 흐름 옵션
        - 페이로드(시그니처 설정 옵션)
            - 악성 패킷을 탐지하는 옵션
            - 시그니처를 지정
            - content: 매칭할 문자열 지정 --> 시그니처.
            - nocase: 대소문자 고려 X
            - pcre: 문자열로 표현하기 어려운 것들을 정규 표현식을 이용하여 정의 할 경우 사용

            - 문자열 지정 content: “administrator”;
            - 숫자 지정 content: “|121212|”;
            - 정규표현식 지정 pcre: “/^select/”; 
                - “/^select/”; 검색할 문자열 중 가장 앞에 위치한 select 문자가 있는 경우 매치
                - 7계층 메시지의 악성코드도 탐지 가능 --> 메시지에 특정한 패턴(시그니처)가 존재할 것이다!

            - 옵션 Page 20
                - **content**: 매칭할 문자열 지정

                - nocase: 대소문자 구별하지 않음

                - offset:
                    - 매칭할 문자열의 위치 지정. 패턴을 검사할 시작 위치로, 첫 번째 바이트 위치가 0부터 시작.

                - depth: offset부터 몇 바이트까지 검사할 것인지 지정.

                - distance: 
                    - Content 옵션값 이후 탐색할 위치 지정
                    - 이전 content 패턴이 매치된 경우, 매치된 바이트로부터 몇 바이트 떨어진 위치에서 다음 content를 검사 할 것인지 지정

                - within: 
                    - Content옵션값 이후의 탐색할 범위를 지정
                    - distance부터 몇 바이트 범위 내에서 지정된 패턴을 검사할 것인지 지정

                - **pere**: 
                    - 문자열로 표현하기 어려운 것들을 **정규 표현식**을 이용하여 정의 할 경우 사용
                    - pcre: “/^select/”; 
        - HTTP 관련 옵션 등
    - 옵션들은 ;(세미콜론)으로 구분




- 실습 시나리오: ftp 접속 시 관리자 로그인을 탐지하라.

1. local rule setting
    - alert tcp any any -> any 21 (msg:"FTP admin login"; content:"user msfadmin"; nocase;)




2. SecurityOnion-snort 탐지 정책 생성
    - 정삭적인 접근과 비정상적 접근을 분류하기 위해
    - Threshold : 동일한 특정 패킷이 관리자 설정한 시간안 에 일정 수가 발견이 되면 경고 알림을 출력해주는 것. 

    - threshold:type [limit,threshold,both], track [by_src, by_dst], count [몇초], seconds [횟수]

    - threshhold에 따라 로그 개수가 달라진다.

        - limit : count 동안 횟 수번째 트래픽까지 탐지
        - threshold : 횟수 마다 계속 탐지
        - both : count 동안 횟수 만큼 트래픽이 탐지 될 시1번 만 탐지
        - by_src : 출발지 패킷만 해당
        - by_dst : 도착지 패킷만 해당

    - 사용 예시
        - threshold:type threshold,count 100, seconds 2; 
            - 로그 발생 기준: 패킷양
            - 로그 발생 예시: 
                - 2초내에 패킷 100개: 로그 1개
                - 2초내에 패킷 200개: 로그 2개
                - 4초내에 패킷 400개: 로그 4개
                - 패킷 양만 충족하면 로그 생성

        - threshold:type limit,count 100, seconds 2; 
            - 로그 발생 기준: 임계시간
            - 로그 발생 예시
                - 2초내에 패킷 100개: 로그 1개
                - 2초내에 패킷 200개: 로그 2개
                - 4초내에 패킷 400개: 로그 2개
                - 임계시간 내에 일정 패킷양 이상이면 로그 생성. 임계시간이 넘어가기 전에 1개만 만들어진다. 특정 시간마다 발생
        - threshold:type both,count 100, seconds 2;
            - 로그 발생 기준: IP
            - 로그 발생 예시
                - 2초내에 패킷 100개 : 로그 1개
                - 2초내에 패킷 200개 : 로그 1개
                - 4초내에 패킷 400개 : 로그 1개
                - 발생 여부만 체크
        
    - 사용 예제
        - alert tcp 192.168.10.20/32 23 -> any any (msg:"Telnet Password Crackinh Attack Detection"; sid:10005; content:"login incorrect"; nocase; threshold: type threshold, track by_src, count 3, seconds 20;) 



3. log 확인
    - tail -f /var/log/snort/snort.alert.fast --> 파일로 저장된 로그 확인
    - cd /var/log/snort에서 snort.alert.fast파일이 텍스트화된 로그 파일




4. 포트 스캔 탐지

- Stealth Scan
- FIN: alert tcp any any -> any any(msg:"Null Port Scan Attack";sid:100006;flags: F; threshold: type threshold, track by_src, count 3, seconds 20;)
- Xmas: alert tcp any any -> any any(msg:"Null Port Scan Attack";sid:100006;flags: UPF;)
- Null: alert tcp any any -> any any(msg:"Null Port Scan Attack";sid:100006;flags: !UAPRSF;)

- alert tcp any any -> any any(msg:"Null Port Scan Attack";sid:100006;flags: !UAPRSF;)
    - content 대신 **헤더의 필드** 사용 가능.

- detection 방법
    1. IP, Port 번호
    2. content
    3. content: threshold
    4. 프로토콜 헤더 필드 값



## DDoS(Distributed Deny of Service) Attack
- 과도한 트래픽을 공격대상에게 전송하여 서비스를 불가하게 하는 공격 기법
- 과도한 트래픽 또는 부하를 발생시켜 정상적인 통신이 불가능하게 만드는 통신 유형

- 실습은 3,4계층에 기반한 DDoS공격

- 공격자가 서버를 공격해 다운시키면 클라이언트도 서비스를 받을 수 없어 피해를 받게 된다.
- 정보 습득이 목적이 아니라 서비스를 못하게 하도록 막는 공격

### 1. 통신의 기본 3요소
1. 전송 매체(회선)
    - End-to-End 연결 통로
    - 각 전송 매체 별로 수용 가능한 대역폭을 보유

2. 정보원(송수신자)
    - End-to-End
    - End-to-End 연결 중계 장비
    - 각각 처리할 수 있는 최대 성능 존재
    - 최대 성능은 CPU/메모리 등 장착되는 부품에 따라 달라짐

3. 프로토콜
    - 통신 규약
    - 정상적인 통신을 위해 미리 정의된 규약에 맞춰 데이터 송수신

### 2. DDoD 공격 원리
- 전송 매체 별 **자신이 수용 가능한 대역폭 이상의 트래릭이 전송될 경우**, 전송된 트래픽을 수용하지 못하여 정상적인 통신이 불가능해짐

- 각 정보원이 처리 가능한 성능 이상의 요청이 발생할 경우 이를 처리하지 못하여 정상적인 통신이 불가능해진다. 

- 1초에 100만개의 요청을 처리하는 서버에 1초에 500만개의 요청을 보내는 것. 

- 대역폭을 늘리면 DoS 공격은 피할 수 있다.
    - CPU/RAM, 회선 대역폭을 늘려 DoS 공격을 파훼
    - 더 개량된 DDoS 공격이 발생
    - DoS 공격은 공격자와 타깃이 1대1
    - DDoS 공격은 공격자가 다수. ex 10대 1

    ### DoS
    - 특정 공격 PC 또는 서버 1대에서 공격 대상 서버 1대로 과도한 트래픽 또는 패킷을 전송하는 1:1 형태

    - CPU/RAM 성능이나 대역폭을 늘려 대처할 수 있었다.

    ### DDoS(분산서비스거부공격)
    - 서버와 차단 장비의 성능이 높아짐에 따라 DoS와 같은 1:1 공격은 더 큰 효과를 낼 수 없게 되었음
    - 공격 성능을 증대 시키기 위해 탄생한 것
    - 악성코드에 감염된 여러 대의 좀비들을 이용하여 동시에 공격하므로 N:1 형태를 뜀

    - 좀비PC들을 지속적으로 패치하고 관리하는 C&C 서버가 존재
    - 이 서버를 관리하는 공격자가 존재

### 3. DDoS 공격 목적
- 금전 요구 또는 개인적 원한, 경쟁사에 의한 공격/청부, 핵티비즘(사상 전달을 위해 공공기관을 공격), 전쟁 전 사회적 혼란을 야기하기 위한 목적 등이 존재한다.

## DDoS 공격, 어렵지 않다!
- 네트워크 통신에서는 요청에 대한 응답을 해주어야 하기 때문에, 빠른 시간 내 무수히 많은 요청을 날려 마비시킬 수 있다!

- 3계층 --> Ping of Death
- hping --icmp --rand-source [IP] -d 65000 --flood
    - 6만5천byte의 패킷을 지속적으로 전송. 
    - 패킷 분할(단편화)의 특징을 이용한 공격
    - 공격자도 지속적으로 단편화를 진행하므로 다운될 위험이 있다.
    - **실습할 때 2000정도로 설정하기!!**

### SYN Flooding Attack
- 공격 실습 후 시그니처 파악하기
- 포트 스캔 공격과의 차이점 찾기

- hping3 --rand-source 192.168.10.20 –p 80 –S --flood
    - --rand-source: 송신지 IP 랜덤 생성
    - 192.168.10.20: 수신지 IP
    - -p 80: 포트 80번으로
    - -S: TCP패킷의 SYN 플래그 설정
    - --flood: CPU를 최대한 활용하며 제한 없이 계속 보냄

- 80번 포트를 열어 뒀으면 SIP로 ACK,SYN을 보냄
- SIP는 가상 주소로 실제로 존재하지 않는 IP.
- ACK를 기다리면서 계속 재전송하며 자신의 자원을 낭비시킨다.

- 탐지 룰 생성
    - 송신지 주소는 랜덤
    - 송신지 포트 번호도 랜덤
    - 수신지도 정확한 정보 상관 없음
    - SYN 요청이 무수히 많다. --> 시그니처
    
    - alert any any -> any 80 (msg:"TCP SYN Flooding Attack"; sid:100008;flags:S; threshold: type limit, track by_dst, count 10, seconds 1;)

### Fragmentation Flooding Attack
- 문자열 65000 바이트로 네트워크에 ping전송
- hping3 --icmp --rand-source 192.168.10.20 –d 2000 --flood #컴퓨터 다운위험으로 2000개, 즉시 종료

- ICMP echo request가 빠르게 전송된다.
- 송신지 IP 주소가 계속 바뀐다.
- 단편화가 계속 되는 것을 확인할 수 있음
- 이 메시지의 메시지 Byte가 모두 58로 구성되어 있는 것을 확인할 수 있다.
    - 시그니처!
    - content: "|5858585858|";

# 암호화 인증서

1. 공개키(비대칭키) 암호화
    - 공개키는 USB, 또는 이메일 등으로 보낼 수 있다.
        - 기밀성
    - 공개키로 복호화할 수도 있다!
        - 개인키로 암호화(서명 -> 인증)
        - 신뢰성
    - Hash Function & Digest. 
        - 개인키로 서명(해시값 = Digest으로 변환 후 서명)
        - 전자 서명된 원본 데이터를 전송
        - Bob은 Alice의 공용키를 이용하여 전자 서명을 해독한 후 Alice의 신원을 확인, Digest를 꺼낸다.
        - 받은 데이터를 동일한 함수를 이용하여 해시하여 Digest 생성. 
        - 이 두 Digest를 비교하여 정확히 일치하면 무결성 보장.





### SSL

- 서버와 클라이언트가 주고 받는 방식
    1. 서버가 클라이언트에게 자신의 공개키를 전송
    2. 클라이언트가 세션키를 생성하여 서버의 공개키로 암호화 후 전송
    3. 서버가 자신의 비밀키로 복호화하여 세션키(대칭키)를 획득
    4. 클라이언트는 세션키로 암호화 후 데이터 전송
    

- 프리 마스터 키(Pre-Master Key)
    - TLS handshake 과정에서 클라이언트와 서버가 공유하는 임시 비밀값
    - 최종적으로 세션 키(Session Key)를 생성하기 위한 원재료

    1. 클라이언트가 프리 마스터 키를 생성하고, 서버의 공개 키로 암호화해 전송
    2. 서버는 자신의 개인 키로 복호화해서 동일한 프리 마스터 키를 획득
    3. 이후 클라이언트와 서버는 같은 프리 마스터 키를 기반으로 세션 키를 생성하여 암호화, 통신을 시작한다.

    - 같은 세션 키를 안전하게 공유하기 위한 중간 단계
    - - 프리 마스터 키가 유출되면 안전한 세션 키 생성이 불가능해지므로 보안성이 매우 중요
